In [1]:
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import random
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, WebDriverException
import re
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
# Đọc file Excel
df = pd.read_excel('D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/crawlcv/IndeedITLink.xlsx')

In [3]:
# Khởi tạo trình duyệt Chrome
options = uc.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
# Nếu cần proxy, bạn có thể thêm:
# options.add_argument('--proxy-server=http://your-proxy-address:port')
driver = uc.Chrome(options=options, version_main=122)

In [4]:
results = []
skipped_links = []

In [5]:
# Hàm định dạng văn bản
def format_text(text):
    # Loại bỏ hoàn toàn các ký tự xuống dòng
    text = text.replace('\n', ' ')
    # Loại bỏ các dấu thừa (như ",," hoặc khoảng trắng thừa)
    text = re.sub(r',+', ',', text)  # Thay nhiều dấu "," liên tiếp thành 1 dấu
    # Chuẩn hóa dấu | trong thông tin liên hệ
    text = re.sub(r'\s*\|\s*', ' | ', text)
    text = re.sub(r'(\|\s*){2,}', ' | ', text)  # Loại bỏ nhiều dấu | liên tiếp
    # Định dạng số điện thoại
    phone_pattern = r'\(?\d{3}\)?\s*-?\s*\d{3}\s*-?\s*\d{4}'
    phone_matches = re.findall(phone_pattern, text)
    for phone in phone_matches:
        phone_clean = re.sub(r'\D', '', phone)
        if len(phone_clean) == 10:
            formatted_phone = f"{phone_clean[:3]}-{phone_clean[3:6]}-{phone_clean[6:]}"
            text = text.replace(phone, formatted_phone)
    # Định dạng email
    email_pattern = r'[\w\.-]+@[\w\.-]+\.\w+'
    email_matches = re.findall(email_pattern, text)
    for email in email_matches:
        text = text.replace(email, email.strip())
    # Định dạng địa chỉ
    address_pattern = r'[A-Za-z\s]+,\s*[A-Za-z\s]+,\s*[A-Za-z\s]+,\s*\d{5}'
    address_matches = re.findall(address_pattern, text)
    for address in address_matches:
        formatted_address = address.replace(',', ', ')
        text = text.replace(address, formatted_address)
    # Loại bỏ khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [6]:
for index, row in df.iterrows():
    category = row['Category']
    resume_link = row['Resume_link']

    try:
        print(f"Đang crawl: {resume_link}")
        driver.get(resume_link)
        time.sleep(random.uniform(2, 5))

        # Kiểm tra CAPTCHA
        captcha_elements = driver.find_elements(By.XPATH, "//*[contains(text(), 'Additional Verification Required')]")
        if captcha_elements:
            print(f"CAPTCHA detected on {resume_link}. Please solve it manually.")
            time.sleep(30)
            captcha_elements = driver.find_elements(By.XPATH, "//*[contains(text(), 'Additional Verification Required')]")
            if captcha_elements:
                print(f"Failed to solve CAPTCHA on {resume_link}. Skipping this link...")
                skipped_links.append(resume_link)
                continue

        # Chờ phần tử chính xuất hiện
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CLASS_NAME, "editor-module"))
        )
        time.sleep(random.uniform(1, 3))
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.uniform(0.5, 1.5))

        # Tìm tab
        tab_buttons = driver.find_elements(By.XPATH, "//button[@role='tab']")
        if not tab_buttons:
            print(f"Không tìm thấy tab trong {resume_link}")
            continue

        tab_container = driver.find_element(By.XPATH, "//div[contains(@id, 'ifl-Tabs-') and contains(@class, 'css-1nf98gr')]")
        if not tab_container:
            print(f"Không tìm thấy tab container trong {resume_link}")
            continue

        tab_container_id = tab_container.get_attribute('id')
        match = re.search(r'ifl-Tabs-:([A-Za-z0-9]+):', tab_container_id)
        if not match:
            print(f"Không thể xác định mã ngẫu nhiên của tab trong {resume_link}")
            continue

        random_code = match.group(1)

        for tab_idx in range(2):
            tab_id = f"ifl-Tabs-{random_code}-ifl-TabPanel-{tab_idx}"

            try:
                tab_elements = driver.find_elements(By.XPATH, f"//div[@id='{tab_id}']")
                if not tab_elements:
                    print(f"Tab {tab_idx} ({tab_id}) không tồn tại trong {resume_link}, bỏ qua...")
                    continue

                if tab_idx >= len(tab_buttons):
                    print(f"Không có button cho tab {tab_idx} trong {resume_link}, bỏ qua...")
                    continue

                tab_button = tab_buttons[tab_idx]
                driver.execute_script("arguments[0].scrollIntoView(true);", tab_button)
                time.sleep(random.uniform(0.5, 1.5))

                actions = ActionChains(driver)
                actions.move_to_element(tab_button).perform()
                time.sleep(random.uniform(0.5, 1))

                WebDriverWait(driver, 30).until(
                    EC.element_to_be_clickable((By.XPATH, f"//button[@id='ifl-Tabs-{random_code}-ifl-Tab-{tab_idx}']"))
                )

                try:
                    tab_button.click()
                except ElementClickInterceptedException:
                    print(f"Không thể nhấp trực tiếp vào tab {tab_idx}, thử dùng JavaScript...")
                    driver.execute_script("arguments[0].click();", tab_button)

                WebDriverWait(driver, 30).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[@id='{tab_id}']//div[@class='css-u74ql7 eu4oa1w0']"))
                )
                time.sleep(random.uniform(1, 3))

                page_html = driver.page_source
                soup = BeautifulSoup(page_html, 'html.parser')

                with open(f"debug_html_{index}_tab_{tab_idx}.html", "w", encoding="utf-8") as f:
                    f.write(page_html)

                editor_module = soup.find('div', class_='editor-module css-1a5am7x eu4oa1w0')
                if not editor_module:
                    print(f"Không tìm thấy editor-module trong {resume_link} (Tab {tab_idx})")
                    continue

                tab_container = editor_module.find('div', class_='css-1nf98gr e1v394w80')
                if not tab_container:
                    print(f"Không tìm thấy css-1nf98gr trong {resume_link} (Tab {tab_idx})")
                    continue

                content_container = tab_container.find('div', class_='css-1xdhyk6 e1iof0s10')
                if not content_container:
                    print(f"Không tìm thấy css-1xdhyk6 trong {resume_link} (Tab {tab_idx})")
                    continue

                tab = content_container.find('div', id=tab_id)
                if not tab:
                    print(f"Không tìm thấy tab {tab_id} trong {resume_link}")
                    continue

                text_parts = []
                seen_texts = set()

                outer_content = tab.find('div', class_='css-u74ql7 eu4oa1w0')
                if not outer_content:
                    print(f"Không tìm thấy css-u74ql7 trong tab {tab_id} của {resume_link}")
                    continue

                content_wrapper = outer_content.find('div', class_='css-q5hitg eu4oa1w0')
                if not content_wrapper:
                    print(f"Không tìm thấy css-q5hitg trong tab {tab_id} của {resume_link}")
                    continue

                inner_content = content_wrapper.find('div', class_='css-u74ql7 eu4oa1w0')
                if not inner_content:
                    print(f"Không tìm thấy css-u74ql7 (nội bộ) trong tab {tab_id} của {resume_link}")
                    continue

                content_sections = inner_content.find_all('div', class_='css-1v9pyvw eu4oa1w0')

                # Bắt đầu phân tích nội dung từng section
                for section in content_sections:
                    h3 = section.find('h3', class_='css-19tlfee')
                    if h3:
                        text = format_text(h3.get_text(strip=True))
                        if text and text not in seen_texts:
                            text_parts.append(text)
                            text_parts.append("")
                            seen_texts.add(text)

                    contact_div = section.find('div', class_='css-vjn8gb')
                    if contact_div:
                        raw_text = contact_div.get_text(separator=" | ")
                        parts = [part.strip() for part in raw_text.split(" | ") if part.strip()]
                        text = format_text(" | ".join(parts))
                        if text and text not in seen_texts:
                            text_parts.append(text)
                            text_parts.append("")
                            seen_texts.add(text)

                    h4 = section.find('h4', class_='css-obxj93')
                    if h4:
                        section_title = format_text(h4.get_text(strip=True))
                        if section_title and section_title not in seen_texts:
                            text_parts.append(section_title)
                            text_parts.append("")
                            seen_texts.add(section_title)

                        # Lấy nội dung tùy theo tiêu đề
                        if section_title == "Summary":
                            content = section.find('div', class_='css-vjn8gb')
                            if content:
                                text = format_text(content.get_text(separator=" ", strip=True))
                                if text and text not in seen_texts:
                                    text_parts.append(text)
                                    text_parts.append("")
                                    seen_texts.add(text)

                        elif section_title == "Education":
                            education_div = section.find('div', class_='css-u74ql7')
                            if education_div:
                                items = education_div.find_all(['div', 'span'], class_=['css-1avvf63', 'css-yev86y'])
                                for item in items:
                                    text = format_text(item.get_text(strip=True))
                                    if text and text not in seen_texts:
                                        text_parts.append(text)
                                        seen_texts.add(text)
                                text_parts.append("")

                        elif section_title == "Experience":
                            experience_divs = section.find_all('div', class_='css-hxk5yu')
                            for experience_div in experience_divs:
                                block = experience_div.find('div', class_='css-bev4h3')
                                if block:
                                    h5s = block.find_all('h5', class_='css-1s9glto')
                                    for h5 in h5s:
                                        text = format_text(h5.get_text(separator=", ", strip=True))
                                        if text and text not in seen_texts:
                                            text_parts.append(text)
                                            seen_texts.add(text)
                                ul = experience_div.find('ul', class_='css-hzkjd0')
                                if ul:
                                    for li in ul.find_all('li', class_='css-omaws3'):
                                        text = format_text(li.get_text(strip=True))
                                        if text and text not in seen_texts:
                                            text_parts.append(text)
                                            seen_texts.add(text)
                            text_parts.append("")

                        elif section_title == "Certifications" or section_title == "Skills":
                            ul = section.find('ul', class_='css-hzkjd0')
                            if ul:
                                for li in ul.find_all('li', class_='css-omaws3'):
                                    text = format_text(li.get_text(strip=True))
                                    if text and text not in seen_texts:
                                        text_parts.append(text)
                                        seen_texts.add(text)
                                text_parts.append("")

                # Sau khi duyệt xong section, lưu kết quả
                if text_parts:
                    resume_str = "\n".join(part for part in text_parts if part).strip()
                    results.append({
                        'Category': category,
                        'Resume': resume_str
                    })
                    print(f"Đã crawl Tab {tab_idx} của {resume_link}")

            except (TimeoutException, ElementClickInterceptedException) as e:
                print(f"Lỗi khi xử lý tab {tab_idx} của {resume_link}: {str(e)}")
                continue

        time.sleep(random.uniform(5, 10))

    except Exception as e:
        print(f"Lỗi khi crawl {resume_link}: {str(e)}")
        continue


Đang crawl: https://www.indeed.com/career-advice/resume-samples/android-developer
Đã crawl Tab 0 của https://www.indeed.com/career-advice/resume-samples/android-developer
Đã crawl Tab 1 của https://www.indeed.com/career-advice/resume-samples/android-developer
Đang crawl: https://www.indeed.com/career-advice/resume-samples/devops-engineer
Đã crawl Tab 0 của https://www.indeed.com/career-advice/resume-samples/devops-engineer
Đã crawl Tab 1 của https://www.indeed.com/career-advice/resume-samples/devops-engineer
Đang crawl: https://www.indeed.com/career-advice/resume-samples/desktop-support
Đã crawl Tab 0 của https://www.indeed.com/career-advice/resume-samples/desktop-support
Đã crawl Tab 1 của https://www.indeed.com/career-advice/resume-samples/desktop-support
Đang crawl: https://www.indeed.com/career-advice/resume-samples/embedded-software-engineer
Đã crawl Tab 0 của https://www.indeed.com/career-advice/resume-samples/embedded-software-engineer
Đã crawl Tab 1 của https://www.indeed.com/c

In [7]:
# Đóng trình duyệt
driver.quit()

In [8]:
# Chuyển dữ liệu thành DataFrame
df_output = pd.DataFrame(results)

In [9]:
df_output.head(20)

,Category,Resume
0,INFORMATION TECHNOLOGY,"Sabrina Chesterfield\nBuffalo, NY | 716-555-01..."
1,INFORMATION TECHNOLOGY,"Darnell Roberts\nNew York, NY | 212-555-0164 |..."
2,INFORMATION TECHNOLOGY,"Ebony Moore\nSecaucus, NJ | 201-555-0187 | emo..."
3,INFORMATION TECHNOLOGY,"Malik Rabb\nSeattle, WA | 206-555-0194 | mrabb..."
4,INFORMATION TECHNOLOGY,"Ivy Haddington\nHouston, TX | 587-555-0176 | i..."
5,INFORMATION TECHNOLOGY,"Ebony Moore\nLittle Rock, AR | 873-555-0156 | ..."
6,INFORMATION TECHNOLOGY,"Cody Fredrickson\nIndianapolis, IN | 317-555-0..."
7,INFORMATION TECHNOLOGY,"Leo Lopez\nHartford, Connecticut | 916-555-017..."
8,INFORMATION TECHNOLOGY,"Cody Fredrickson\nPortland, OR | 503-555-0187 ..."
9,INFORMATION TECHNOLOGY,"Ivy Haddington\nGainesville, FL | 352-555-3985..."


In [10]:
import os
# Đường dẫn file đầu ra
output_dir = "D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/crawlcv"
output_file = os.path.join(output_dir, "finalindeeditlink.xlsx")

In [11]:
# Lưu vào file CSV
df_output.to_excel(output_file, index=False)
print(f"Dữ liệu đã được lưu vào file: {output_file}")
print(f"Tổng số bản ghi crawl được: {len(df_output)}")

Dữ liệu đã được lưu vào file: D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/crawlcv\finalindeeditlink.xlsx
Tổng số bản ghi crawl được: 32
